In [1]:
import re
import pymysql

def is_valid_email(email):
    """
    Checks if the provided string is a valid email address.
    """
    # Regular expression pattern for email validation
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return re.match(pattern, email) is not None

def fetch_user_email(username):
    """
    Fetches the user_email associated with the provided username or email from the database.
    """
    connection = None  # Initialize connection variable
    try:
        # Check if the provided username is an email address
        if is_valid_email(username):
            return username

        # Connect to the MySQL database
        connection = pymysql.connect(host='127.0.0.1',
                                     port=3306,
                                     user='root',
                                     password='',
                                     db='bikerental')

        # Create a cursor object
        with connection.cursor() as cursor:
            # Execute a SQL query to retrieve the user_email
            query = "SELECT EmailId FROM tblusers WHERE FullName = %s"
            cursor.execute(query, (username,))
            
            # Fetch the user_email
            result = cursor.fetchone()
            if result:
                user_email = result[0]
                return user_email  # Return the user email if found

            # If no user email is found
            print("User not found.")
            return None

    except pymysql.Error as e:
        print(f"Error: {e}")

    finally:
        # Close the connection
        if connection:
            connection.close()

    print("An error occurred while fetching user email.")
    return None

# # Example usage:
# # Replace 'username' with actual login credentials
# username = 'admin@example.com'  # Assume the user provides an email address
# user_email = fetch_user_email(username)
# if user_email:
#     print("User Email:", user_email)



In [2]:
# Example usage:
# Replace 'username' with actual login credentials
username = 'Tom K'  # Assume the user provides a username
user_email = fetch_user_email(username)
if user_email:
    print("User Email:", user_email)


User Email: test@gmail.com


In [3]:
import pandas as pd
import pymysql
import random

def preprocess_data(user_email):
    """
    Connects to a MySQL database, retrieves vehicle data associated with the user,
    preprocesses it, and returns a pandas DataFrame.
    """

    # Connect to the MySQL database
    connection = pymysql.connect(host='127.0.0.1',
                                 port=3306,
                                 user='root',
                                 password='',
                                 db='bikerental')

    # Create a cursor object
    cursor = connection.cursor()

    # Execute a SQL query to retrieve the relevant features based on user's email
    query = f"SELECT v.id, v.VehiclesTitle, v.VehiclesBrand, v.PricePerDay, v.FuelType, v.ModelYear, v.SeatingCapacity, \
                     v.AirConditioner, v.PowerDoorLocks, v.AntiLockBrakingSystem, v.BrakeAssist, v.PowerSteering, \
                     v.DriverAirbag, v.PassengerAirbag, v.PowerWindows, v.CDPlayer, v.CentralLocking, v.CrashSensor, \
                     v.LeatherSeats \
              FROM tblvehicles v \
              JOIN tblbooking b ON v.id = b.VehicleId \
              WHERE b.userEmail = '{user_email}'"
              
    cursor.execute(query)

    # Fetch all the rows as a list of tuples
    rows = cursor.fetchall()

    # Create a pandas DataFrame from the list of tuples
    df = pd.DataFrame(rows, columns=[
                      'id', 'VehiclesTitle', 'VehiclesBrand', 'PricePerDay', 'FuelType', 'ModelYear', 'SeatingCapacity',
                      'AirConditioner', 'PowerDoorLocks', 'AntiLockBrakingSystem', 'BrakeAssist', 'PowerSteering',
                      'DriverAirbag', 'PassengerAirbag', 'PowerWindows', 'CDPlayer', 'CentralLocking', 'CrashSensor',
                      'LeatherSeats'])

    # Preprocess data (example: handle missing values)
    df.fillna(random.choice([0, 1]), inplace=True)  # Replace null values with random choice (0 or 1)

    # Convert boolean columns to integers (optional)
    bool_columns = ['AirConditioner', 'PowerDoorLocks', 'AntiLockBrakingSystem', 'BrakeAssist', 'PowerSteering',
                    'DriverAirbag', 'PassengerAirbag', 'PowerWindows', 'CDPlayer', 'CentralLocking', 'CrashSensor',
                    'LeatherSeats']
    df[bool_columns] = df[bool_columns].astype(int)

    # Close the cursor and the connection
    cursor.close()
    connection.close()

    # Return the preprocessed DataFrame
    return df

In [4]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

def recommendation_function(preprocessed_data):
    # Define the fitness function
    def evaluate(individual):
        score = 0
        for i in range(len(individual)):
            if individual[i] == 1:
                # Assuming 'PricePerDay' is a relevant feature for score calculation
                score += preprocessed_data.loc[i, 'PricePerDay']
        return (score,)
    
    # Create the individual class and population
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(preprocessed_data))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Initialize population
    population = toolbox.population(n=50)
    
    # Run the genetic algorithm
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)
    
    # Return the best individual
    best_individual = hof[0]
    recommendation = [preprocessed_data.loc[i] for i, val in enumerate(best_individual) if val == 1]
    
    # Return only the top 10 recommendations
    return recommendation


import json


In [5]:
import json

def format_recommendations(recommendations):
    formatted_recommendations = []
    for rec in recommendations[:5]:
        formatted_rec = {
            "VehicleId": int(rec['id']),
            "VehiclesTitle": rec['VehiclesTitle'],
            "PricePerDay(Npr)": round(float(rec['PricePerDay']), 2),
            "AirConditioner": 'Yes' if rec['AirConditioner'] == 1 else 'No',
            "AntiLockBrakingSystem": 'Yes' if rec['AntiLockBrakingSystem'] == 1 else 'No',
            "DriverAirbag": 'Yes' if rec['DriverAirbag'] == 1 else 'No',
            "PassengerAirbag": 'Yes' if rec['PassengerAirbag'] == 1 else 'No',
            "LeatherSeats": 'Yes' if rec['LeatherSeats'] == 1 else 'No'
        }
        formatted_recommendations.append(formatted_rec)
    return formatted_recommendations


In [6]:
def get_recommendation(user_email):
    data = preprocess_data(user_email)
    recommendations = recommendation_function(data)
    return recommendations

# Call get_recommendation to get recommendations
recommendations = get_recommendation(user_email)

# Format recommendations and print JSON output
formatted_recommendations = format_recommendations(recommendations)
json_output = json.dumps(formatted_recommendations)
print(json_output)

C:\Users\Ripple\AppData\Local\Temp\ipykernel_287428\3577071832.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(random.choice([0, 1]), inplace=True)  # Replace null values with random choice (0 or 1)


gen	nevals	avg    	min	max 
0  	50    	3167.74	0  	7058
1  	38    	5818.22	563	7058
2  	39    	6921.14	6199	7058
3  	36    	7046.74	6495	7058
4  	36    	7029.56	6199	7058
5  	34    	6934.02	1422	7058
6  	35    	6928.1 	1422	7058
7  	45    	6934.02	1422	7058
8  	41    	7046.74	6495	7058
9  	38    	6945.28	1422	7058
10 	36    	6916.84	1422	7058
11 	40    	6922.76	1422	7058
12 	40    	7046.74	6495	7058
13 	34    	7058   	7058	7058
14 	40    	7058   	7058	7058
15 	38    	7040.82	6199	7058
16 	41    	7058   	7058	7058
17 	36    	6804.12	1422	7058
18 	39    	7058   	7058	7058
19 	34    	7058   	7058	7058
20 	36    	7058   	7058	7058
21 	41    	6821.3 	859 	7058
22 	38    	7058   	7058	7058
23 	27    	6821.3 	1422	7058
24 	43    	6934.02	859 	7058
25 	40    	7023.64	6199	7058
26 	32    	6934.02	1422	7058
27 	38    	7040.82	6199	7058
28 	44    	6934.02	1422	7058
29 	40    	7046.74	6495	7058
30 	38    	6945.28	1422	7058
31 	44    	6945.28	1422	7058
32 	42    	7040.82	6199	7058
33 	35    	7006.4

In [7]:
# Format recommendations and print JSON output
formatted_recommendations = format_recommendations(recommendations)
json_output = json.dumps(formatted_recommendations)
print(json_output)

[{"VehicleId": 2, "VehiclesTitle": "RS200", "PricePerDay(Npr)": 859.0, "AirConditioner": "Yes", "AntiLockBrakingSystem": "Yes", "DriverAirbag": "Yes", "PassengerAirbag": "Yes", "LeatherSeats": "Yes"}, {"VehicleId": 3, "VehiclesTitle": "R1", "PricePerDay(Npr)": 563.0, "AirConditioner": "Yes", "AntiLockBrakingSystem": "Yes", "DriverAirbag": "Yes", "PassengerAirbag": "Yes", "LeatherSeats": "Yes"}, {"VehicleId": 4, "VehiclesTitle": "Duke390", "PricePerDay(Npr)": 5636.0, "AirConditioner": "Yes", "AntiLockBrakingSystem": "Yes", "DriverAirbag": "Yes", "PassengerAirbag": "Yes", "LeatherSeats": "Yes"}]


In [8]:
import re
import pymysql
import pandas as pd
import random
import numpy as np
from deap import base, creator, tools, algorithms
import json

def is_valid_email(email):
    """
    Checks if the provided string is a valid email address.
    """
    # Regular expression pattern for email validation
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return re.match(pattern, email) is not None

def fetch_user_email(username):
    """
    Fetches the user_email associated with the provided username or email from the database.
    """
    connection = None  # Initialize connection variable
    try:
        # Check if the provided username is an email address
        if is_valid_email(username):
            return username

        # Connect to the MySQL database
        connection = pymysql.connect(host='127.0.0.1',
                                     port=3306,
                                     user='root',
                                     password='',
                                     db='bikerental')

        # Create a cursor object
        with connection.cursor() as cursor:
            # Execute a SQL query to retrieve the user_email
            query = "SELECT EmailId FROM tblusers WHERE FullName = %s"
            cursor.execute(query, (username,))
            
            # Fetch the user_email
            result = cursor.fetchone()
            if result:
                user_email = result[0]
                return user_email  # Return the user email if found

            # If no user email is found
            print("User not found.")
            return None

    except pymysql.Error as e:
        print(f"Error: {e}")

    finally:
        # Close the connection
        if connection:
            connection.close()

    print("An error occurred while fetching user email.")
    return None

def preprocess_data(user_email):
    """
    Connects to a MySQL database, retrieves vehicle data associated with the user,
    preprocesses it, and returns a pandas DataFrame.
    """

    # Connect to the MySQL database
    connection = pymysql.connect(host='127.0.0.1',
                                 port=3306,
                                 user='root',
                                 password='',
                                 db='bikerental')

    # Create a cursor object
    cursor = connection.cursor()

    # Execute a SQL query to retrieve the relevant features based on user's email
    query = f"SELECT v.id, v.VehiclesTitle, v.VehiclesBrand, v.PricePerDay, v.FuelType, v.ModelYear, v.SeatingCapacity, \
                     v.AirConditioner, v.PowerDoorLocks, v.AntiLockBrakingSystem, v.BrakeAssist, v.PowerSteering, \
                     v.DriverAirbag, v.PassengerAirbag, v.PowerWindows, v.CDPlayer, v.CentralLocking, v.CrashSensor, \
                     v.LeatherSeats \
              FROM tblvehicles v \
              JOIN tblbooking b ON v.id = b.VehicleId \
              WHERE b.userEmail = '{user_email}'"
              
    cursor.execute(query)

    # Fetch all the rows as a list of tuples
    rows = cursor.fetchall()

    # Create a pandas DataFrame from the list of tuples
    df = pd.DataFrame(rows, columns=[
                      'id', 'VehiclesTitle', 'VehiclesBrand', 'PricePerDay', 'FuelType', 'ModelYear', 'SeatingCapacity',
                      'AirConditioner', 'PowerDoorLocks', 'AntiLockBrakingSystem', 'BrakeAssist', 'PowerSteering',
                      'DriverAirbag', 'PassengerAirbag', 'PowerWindows', 'CDPlayer', 'CentralLocking', 'CrashSensor',
                      'LeatherSeats'])

    # Preprocess data (example: handle missing values)
    df.fillna(random.choice([0, 1]), inplace=True)  # Replace null values with random choice (0 or 1)

    # Convert boolean columns to integers (optional)
    bool_columns = ['AirConditioner', 'PowerDoorLocks', 'AntiLockBrakingSystem', 'BrakeAssist', 'PowerSteering',
                    'DriverAirbag', 'PassengerAirbag', 'PowerWindows', 'CDPlayer', 'CentralLocking', 'CrashSensor',
                    'LeatherSeats']
    df[bool_columns] = df[bool_columns].astype(int)

    # Close the cursor and the connection
    cursor.close()
    connection.close()

    # Return the preprocessed DataFrame
    return df

def recommendation_function(preprocessed_data):
    # Define the fitness function
    def evaluate(individual):
        score = 0
        for i in range(len(individual)):
            if individual[i] == 1:
                # Assuming 'PricePerDay' is a relevant feature for score calculation
                score += preprocessed_data.loc[i, 'PricePerDay']
        return (score,)
    
    # Create the individual class and population
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(preprocessed_data))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Initialize population
    population = toolbox.population(n=50)
    
    # Run the genetic algorithm
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)
    
    # Return the best individual
    best_individual = hof[0]
    recommendation = [preprocessed_data.loc[i] for i, val in enumerate(best_individual) if val == 1]
    
    # Return only the top 10 recommendations
    return recommendation

def format_recommendations(recommendations):
    formatted_recommendations = []
    for rec in recommendations[:5]:
        formatted_rec = {
            "VehicleId": int(rec['id']),
            "VehiclesTitle": rec['VehiclesTitle'],
            "PricePerDay(Npr)": round(float(rec['PricePerDay']), 2),
            "AirConditioner": 'Yes' if rec['AirConditioner'] == 1 else 'No',
            "AntiLockBrakingSystem": 'Yes' if rec['AntiLockBrakingSystem'] == 1 else 'No',
            "DriverAirbag": 'Yes' if rec['DriverAirbag'] == 1 else 'No',
            "PassengerAirbag": 'Yes' if rec['PassengerAirbag'] == 1 else 'No',
            "LeatherSeats": 'Yes' if rec['LeatherSeats'] == 1 else 'No'
        }
        formatted_recommendations.append(formatted_rec)
    return formatted_recommendations

def get_recommendation():
    username = 'Tom K'  # Assume the user provides a username
    user_email = fetch_user_email(username)
    if user_email:
        data = preprocess_data(user_email)
        recommendations = recommendation_function(data)
        formatted_recommendations = format_recommendations(recommendations)
        return formatted_recommendations
    else:
        return None

# Call get_recommendation to get recommendations
recommendations = get_recommendation()

# Save recommendations in a separate variable for later use
recommendations_variable = recommendations

# Print recommendations (as JSON) for immediate use
print(json.dumps(recommendations))


gen	nevals	avg    	min	max 
0  	50    	2700.84	0  	7058
1  	39    	5132.96	563	7058
2  	42    	6392.14	859	7058
3  	32    	6967.92	6495	7058
4  	39    	6810.04	859 	7058
5  	34    	7058   	7058	7058
6  	35    	6945.28	1422	7058
7  	39    	7058   	7058	7058
8  	30    	6928.1 	1422	7058
9  	39    	6905.58	1422	7058
10 	38    	7058   	7058	7058
11 	38    	7058   	7058	7058
12 	37    	7058   	7058	7058
13 	40    	6945.28	1422	7058
14 	47    	7046.74	6495	7058
15 	33    	7007.04	6199	7058
16 	42    	7023.64	6199	7058
17 	40    	7058   	7058	7058
18 	34    	7046.74	6495	7058
19 	41    	6934.02	1422	7058
20 	33    	7023.64	6199	7058
21 	35    	7029.56	6199	7058
22 	38    	7040.82	6199	7058
23 	42    	7058   	7058	7058
24 	37    	7035.48	6495	7058
25 	41    	7023.64	6199	7058
26 	35    	7040.82	6199	7058
27 	36    	6916.84	1422	7058
28 	40    	6905.58	1422	7058
29 	38    	7058   	7058	7058
30 	38    	7058   	7058	7058
31 	41    	6945.28	1422	7058
32 	41    	7040.82	6199	7058
33 	35    	7023.64

C:\Users\Ripple\AppData\Local\Temp\ipykernel_287428\1050459411.py:99: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(random.choice([0, 1]), inplace=True)  # Replace null values with random choice (0 or 1)
c:\Users\Ripple\AppData\Local\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Ripple\AppData\Local\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("